# Deep Learning for Image Super-resolution: A survey

*文章已经放在资源文件中*，文件位置：resources\literature\super resolution\sr-1.pdf

## Abstract

1. Image Super Resulotion（简称为 SR） 是一个在 CV 领域中提高图像和视频分辨率的技术。
2. SR 一般会分成三类：
- 监督的 SR
- 半监督的 SR
- 领域 SR
3. 这篇文章是为了介绍近几年来，Deep Learning 在 SR 领域的一些前沿的进展。并总结了 SR 上的性能评价指标和 benchmark 数据集。 
4. 讨论了一下未来的发展方向


## Introduction

SR 就是从低分辨率的图片中恢复出高分辨率的图片，是图像处理和 CV 中的一个很重要的领域。SR 的应用领域也很广泛：

- 医疗图像
- 监管和审查

SR 问题是 ill-posed。因为一张低分辨率的图片可以对应多张高分辨率的图片。

    如果一个问题是 well-posed，那么它需要满足：1. 解必须存在。2.解必须唯一。3.解能根据初始状态连续变化。否则这个问题就是 ill-posed。

SR 问题上一些传统典型的解决方法：

- prediction-based
- edge-based
- statistical-based
- patch-based
- sparse-representation

Deep Learning 的方法最近在 SR 问题上实现了 SOTA 的效果：

- CNN-based
- GAN

Deep Learning 中的方法差异主要体现在：

- 不同的网络结构
- 不同的损失函数
- 不同的学习策略

这篇文章主要的贡献点在于：

- 基于 Deep Learning 的视角，总结了 SR 问题上的技术、benchmark 数据集、性能指标
- 从层次和结构化的角度系统性的总结了 SR 问题上 Deep Learning 技术的进展
- 讨论了 SR 问题棉铃的挑战

## Problem Setting and Terminology

### Problem Definitions

SR 问题被定义成：

$$
    I_x = D(I_y; \delta)
$$

$D$ 表示的降低分辨率的函数， $I_y$ 表示高分辨率的图片， $\delta$ 表示函数 $D$ 的参数。通常，我们只有 $I_x$，我们就需要通过 $I_x$ ，恢复出真实的 $I_y$ ，也即使得 $\hat{I_y}$ 尽可能的和 $I_y$ 接近。所以：

$$
    I_y = F(I_x;\theta)
$$

其中 $F$ 是 SR 模型，$\theta$ 是 SR 模型的参数。尽管降低分辨率的过程有很多（人工压缩、传感器噪音、色斑噪音），但是大多数研究人员把 $D$ 看作是降采样：

$$
    D(I_y;\delta) = (I_y) \downarrow_s,{s} \subset \delta
$$

其中 $\downarrow_s$ 被看作是参数为 $s$ 的降采样操作。一般采用的降采样是 *带抗锯齿的双三次插值（bicubic interpolation with antialiasing）*。

还有人把 $D$ 看作是通过 *blur kernel* 处理后加入 *标准差为 $\sigma$ 高斯白噪声* 的结果：

$$
    D(I_y;\delta) = (I_y \otimes \kappa) \downarrow_s + n_{\sigma},\{\kappa, s, \sigma\} \subset \delta
$$

其中 $\kappa$ 是 blur kernel。$n_{\sigma}$ 是高斯白噪声。

SR 的目标函数是：

$$
    \hat{\theta} = arg ~ min_{\theta} ~ L(\hat{I_y}, I_y) + \lambda \phi(\theta)
$$

$\phi(\theta)$ 是正则项

## SR 的数据集

现在已经有很多的 SR 数据集，有些同时提供了 LR 和 HR 图片，而有些之提供了 HR 图片，这时候就需要通过 imresize 获取 LR 图片。图表 1 给出了 SR 采用的数据集：

![dataset-table](https://github.com/mayday-975/Machine-Learning-Literature-Review/blob/main/resources/images/super%20resolution/sr-1/sr-dataset-table.png?raw=true)